# Occupancy ベースの方策勾配法

参考
* [Occupancy-based Policy Gradient: Estimation, Convergence, and Optimality](https://openreview.net/forum?id=Nq8enbbaP2)

強化学習では価値をベースとしたアルゴリズムをよく見ます．
しかし，[RL_Convex_MDP](RL_Convex_MDP.ipynb)で見たように，収益の最大化以外の話題で，しばしば価値では扱えない問題が出てきます．
今回はOccupancy measureをベースとした手法について見てみましょう．

表記
* 有限ホライゾンMDPです．表記は論文に任せます
* 方策空間：$\Pi_{\Theta}=\left\{\pi_\theta: \theta \in \Theta, \theta \in \mathbb{R}^p,\|\theta\| \leq B\right\}$
* 目的関数：$f: \Pi_{\Theta} \rightarrow \mathbb{R}$
* 方策更新：$\theta^{(t+1)}=\operatorname{Proj}_{\Theta}\left(\theta^{(t)}+\eta \nabla f\left(\pi_{\left.\theta^{(t)}\right)}\right)\right)$. ここで，$\nabla f\left(\pi_\theta\right)=\left[\frac{\partial f\left(\pi_\theta\right)}{\partial \theta^P}\right]_{p \in[\mathrm{p}]} \in \mathbb{R}^{\mathrm{p}}$
* 今回は方策が決定的にならない状況を考えます：For all $\pi_\theta \in \Pi_{\Theta}, \max _{s, a}\left\|\nabla \log \pi_\theta(a \mid s)\right\|_{\infty} \leq G$.
* 勾配の定常性は$\left\|G^\eta\left(\pi^{(t)}\right)\right\|$ with $G^\eta\left(\pi^{(t)}\right):=\frac{1}{n}\left(\theta^{(t)}-\theta^{(t+1)}\right)$で評価されます

計算効率の評価について：よくある文献と同様に，計算効率をオラクルに対する呼び出し回数で評価します．これは計算自体を抽象化するため，便利です（書き方は参考になるから見ておこう．）

与えられたデータセット$\mathcal{D}=\{(x, y)\}$と関数クラス$\mathcal{F}$に対して，
* $\operatorname{argmax}_{f \in \mathcal{F}} \mathbb{E}_{\mathcal{D}}[\log f(x)]$を返すオラクル
* $\operatorname{argmin}_{f \in \mathcal{F}} \mathbb{E}_{\mathcal{D}}\left[(f(x)-y)^2\right]$を返すオラクル

の２つを考えます．


## Occupancyの再帰的な更新

収益$J(\pi)=\sum_h \sum_{s_h} d_h^\pi\left(s_h\right) R\left(s_h\right)$について，その収益関数の方策勾配は
$$
\nabla J(\pi)=\sum_h \sum_{s_h} \nabla d_h^\pi\left(s_h\right) R\left(s_h\right)=\sum_h \mathbb{E}_{s_h \sim d_h^\pi}\left[\nabla \log d_h^\pi\left(s_h\right) R\left(s_h\right)\right]
$$
です．すなわち，Occupancyについての勾配$\nabla \log d_h^\pi\left(s_h\right)$が評価できれば方策勾配できそうです．
この勾配は次の再帰的な構造を満たします：

$$
\nabla \log d_h^\pi=\mathbf{E}_{h-1}^\pi\left(\nabla \log \pi+\nabla \log d_{h-1}^\pi\right)
$$

よって，FQIのように，次の最小二乗回帰を考えれば，一般の方策空間についての更新が実現されます：

$$
\nabla \log d_h^\pi=\operatorname{argmin}_{g: \mathcal{S} \rightarrow \mathbb{R}^{\boldsymbol{p}}} \mathbb{E}_\pi\left[\left\|g\left(s_h\right)-\left(\nabla \log \pi\left(a_{h-1} \mid s_{h-1}\right)+\nabla \log d_{h-1}^\pi\left(s_{h-1}\right)\right)\right\|^2\right] .
$$


## 一般の目的関数

このOccupancy basedな手法は一般の目的関数にも利用できます．

$J_F(\pi)=\sum_h F_h\left(d_h^\pi\right)$を考えましょう．ここで，$F_h: \Delta(\mathcal{S}) \rightarrow \mathbb{R}$は一般の関数です．
例えば次の例が考えられます：

* $F_h(d)=-\langle d, \log d\rangle$：エントロピー最大化
* $F_h(d)=-\left\|d-d_h^{\pi_E}\right\|_2^2$：模倣学習

など．一般の関数については，次の勾配を考えることで解決できます：

$$
\nabla J_F(\pi)=\sum_h \mathbb{E}_{s \sim d_h^\pi}\left[\left.\frac{\partial F_h(d)}{\partial d(s)}\right|_{d=d_h^\pi} \nabla \log d_h^\pi(s)\right]
$$

ほぼ上と同様ですが，勾配$\left.\frac{\partial F_h(d)}{\partial d(s)}\right|_{d=d_h^\pi}$が必要になります．これはOccupancy $d^\pi_h$が必要になります．